In [26]:
import pycutest
import numpy as np
import time
import pandas as pd

testes = [
    "ARGLINA", "ARGLINB", "BA-L1SPLS", "BIGGS6", "BROWNAL", "COATING",
    "FLETCHCR", "GAUSS2LS", "GENROSE", "HAHN1LS", "HEART6LS", "HILBERTB",
    "HYDCAR6LS", "LANCZOS1LS", "LANCZOS2LS", "LRIJCNN1", "LUKSAN12LS",
    "LUKSAN16LS", "OSBORNEA", "PALMER1C", "PALMER3C", "PENALTY2", "PENALTY3",
    "QING", "ROSENBR", "STRTCHDV", "TESTQUAD", "THURBERLS", "TRIGON1",
    "TOINTGOR"
]

def armijo_facil(problema,ponto,grad,b):
    passo = 1
    k = 1
    a = 1.0e-4*np.dot(grad,grad) 
    for k in range(100):
        if problema.obj(ponto + passo*grad) <= a*passo + b:
            return passo
        else:
            passo *= 0.5
    return passo


In [27]:
class parameters:
    def __init__(self,function: str):
        self.function = pycutest.import_problem(function)                #deixar so os metodos publicos depois
        self.xk = self.function.x0                                      
        self.old_xk = None                                
        self.grad = self.function.grad(self.xk)                          #sempre teremos uma funcao gradiente para computar
        self.grad_calls = 1                                              #por isso inicializamos a chamada de gradiente como 1
        self.val_calls = 0
        self.old_grad = None
        #--------------------------------Classe fila para Armijo Modificada------------------
        self.fila = Fila(self.objective(self.xk))
        
        #-------------------------------Caso Espectral primeiro passo------------------------
        new_xk = self.xk - armijo_facil(self.function, self.xk, self.grad, self.fila.first()) * self.gradient()
        self.get_new_point(new_xk)
        self.fila.add(self.objective(self.xk))
        
    def get_new_point(self,x):
        self.old_xk = self.xk
        self.xk = x
        self.old_grad = self.grad
        self.grad = self.function.grad(self.xk)
        self.grad_calls += 1
        
    def gradient(self):
        return self.grad
    
    def modified(self):
        if self.fila != None:
            return self.fila.max() 
    
    def objective(self, ponto):
        self.val_calls += 1
        return self.function.obj(ponto)
    
    def salva_fila(self, valor)-> None:
        self.fila.add(valor)

    def variation(self):         #retorna delta de x e delta de y
        return  (self.xk - self.old_xk ,self.grad - self.old_grad)  
                      
class Fila:
    def __init__(self, x: float):
        self.vector = [None] * 10                  #classe fila foi criada a fim de ser usada no gradiente espectral
        self.atual = 1                             #pode ser usada como heuristica nos outros metodos
        self.vector[0] = x                         #mas nao sera feito isso nesse relatorio
    
    def add(self, elem):
        self.vector[self.atual] = elem
        self.atual = (self.atual + 1) % 10         #implementa rotação

    def first(self):
        return self.vector[0]
    
    def max(self):
        elementos_validos = [v for v in self.vector if v is not None]
        return max(elementos_validos) if elementos_validos else None 
 

In [28]:
def sigma(function: parameters)-> float:
    deltax, deltay = function.variation()
    """
    a partir de agora e feito o quadrados minimos
    de uma unica variavel
    com a aproximação da matriz quasi newton
    """
    skyk = np.dot(deltax,deltay)
    if skyk > 0.0:                                      
        sigma = skyk / np.dot(deltax,deltax)
    else: 
        sigma = 1.0e-4 * np.linalg.norm(function.grad, ord= np.inf) / max(1.0, np.linalg.norm(function.xk, ord = np.inf))
    return max(1.0e-30, min(sigma, 1.0e30))


In [29]:

def Armijo(function: parameters, passo: float, dk, a, b) ->bool:
    aux = function.objective(function.xk + passo*dk)                  #utilizar armijo so para verificar                        
    if (aux <= passo*a + b):                                          #se o passo e bom ou nao
        function.fila.add(aux)
        return True
    return False

#--------------------------------------------------------------------------------------------------
#Busca usada
def step_parameter(parameters: parameters, direction) -> float:
    M = parameters.fila.max()
    passo = 1.0
    aux = 1e-4*np.dot(parameters.grad,direction)                            #parametro eta = 1e-4 referencia relatorio
    for j in range(100):                                                    #limite j = 100 dificilmente chega nele, um numero exagerado de grande
        if Armijo(parameters, passo,direction,aux,M):                       #explicação detalhada esta no relatorio
            return passo
        else:
            passo = passo*0.5                                               #parametro beta que está no relatorio a referencia
    return passo                                                            #mas qualquer numero entre 0,5 a 0,8 converge bem

def minimize(function: parameters, tolerance = 1e-5):            
    maximum_iterations = int(1e5)
    start = time.process_time()                                         
    for iteration in range(maximum_iterations):
        if np.linalg.norm(function.gradient(), ord= np.inf) < tolerance:
            end = time.process_time()
            tempo = end - start
            return {
                "val_calls": function.val_calls,
                "grad_calls": function.grad_calls,
                "tempo": tempo
            }
        else:
            dk = -function.gradient()/sigma(function)
            step = step_parameter(function,dk)
            function.get_new_point(function.xk + step*dk)
    # Retorna negativo para indicar falha, se não convergir em máximo de iterações
    return {
        "val_calls": -function.val_calls,
        "grad_calls": -function.grad_calls,
        "tempo": 0.0
    }

In [30]:
espc = {i: minimize(parameters(i)) for i in testes[0:1]}
df = pd.DataFrame(espc).T

df

,val_calls,grad_calls,tempo
ARGLINA,16.0,4.0,0.003287
